<a href="https://colab.research.google.com/github/PBHPBH/Machine_Learning/blob/NLP/7_%EC%9B%8C%EB%93%9C_%EC%9E%84%EB%B2%A0%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk
!pip install JPype1
!pip install rhinoMorph

In [ ]:
#Word2Vec – 학습할 텍스트 읽기
import os.path
embedding_dim = 50 # 임베딩 차원수 설정
filepath = '/content/gdrive/My Drive/pytest/'
os.chdir(filepath) # 경로 설정
print("Current Directory:", os.getcwd())
with open('wiki_test.txt', 'r', encoding='utf-8') as f: # 테스트용 파일 읽기
    data = f.read()

Current Directory: /content/gdrive/My Drive/pytest


In [ ]:
#Word2Vec – 문장단위 분리 및 형태소분석기 기동
import rhinoMorph
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
sent_data = sent_tokenize(data) # 문장 단위 분리
rn = rhinoMorph.startRhino() # RHINO 기동
print('type:', type(sent_data))
print('length:', len(sent_data)) # 전체 문장의 개수
print('sentence sample:', sent_data[:20]) # 형태소 분석 전 모습

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
filepath:  /usr/local/lib/python3.7/dist-packages
classpath:  /usr/local/lib/python3.7/dist-packages/rhinoMorph/lib/rhino.jar
RHINO started!
type: <class 'list'>
length: 11977
sentence sample: ['<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">\n지미 카터\n\n제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39번째 대통령 (1977년 ~ 1981년)이다.', '지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.', '조지아 공과대학교를 졸업하였다.', '그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다.', '1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다.', '그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.', '1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다.', '대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다.', '조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.', '1976년 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워, 포드를 누르고 당선되었다.', '카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.', '카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트

In [ ]:
#Word2Vec – 텍스트의 형태소 분석
total_lines = len(sent_data)
cnt = 0
with open(filepath+'word2vec/wiki202003_nationalcorpus_naverratings_morphed.txt', 'w', encoding='utf-8') as f:
    for data_each in sent_data:
        morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each, pos=['NNG', 'NNP', 'NP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
        joined_data_each = ' '.join(morphed_data_each)
        if joined_data_each:
            f.write(joined_data_each + '\n')
        cnt += 1
        if (cnt % 1000) == 0: # 진행 정도 확인을 위해 1000번째 문장마다 확인
            print(round(cnt/total_lines * 100, 3), '%')
print('Morphological Analysis Completed.')

8.349 %
16.699 %
25.048 %
33.397 %
41.747 %
50.096 %
58.445 %
66.795 %
75.144 %
83.493 %
91.843 %
Morphological Analysis Completed.


In [ ]:
#Word2Vec – 형태소 분석 결과를 읽어 리스트로 만들기
def read_data(filename, encoding='utf-8'): # 읽기 함수 정의
    with open(filename, 'r', encoding=encoding) as f:
        data = [line.split(' ') for line in f.read().splitlines()]
    return data
data = read_data(filepath + 'word2vec/wiki202003_nationalcorpus_naverratings_morphed.txt', 'utf-8')
print(len(data))
print(type(data))
print(data[:3])

11976
<class 'list'>
[['URL', '미', '카터', '미', '카터', '제임스', '얼', '미', '카터', '주니어', '민주당', '출신', '미국', '대통령'], ['미', '카터', '조지아', '섬터', '카운티', '마을', '태어나'], ['조지아', '공과대학', '교', '졸업']]


In [ ]:
filepath

'/content/gdrive/My Drive/pytest/'

In [ ]:
#Word2Vec – 임베딩 구성
from gensim.models import Word2Vec
os.chdir(filepath+'word2vec/')
# size: 벡터의 차원, window: 컨텍스트 윈도우의 크기, min_count: 단어 최소 빈도, workers: 학습을 위한 프로세스 수, sg: 0은 CBOW, 1은 skip-gram
model = Word2Vec(sentences=data, size=embedding_dim, window=10, min_count=5, workers=4, sg=1)
model.save('embedding_window10_mincnt5_skipgram.model')
print('Completed.')

Completed.


In [ ]:
#Word2Vec – 임베딩 값 저장
words = list(model.wv.vocab)
with open('embedding_window10_mincnt5_skipgram.txt', 'w') as f:
    for word in words:
        data = model.wv[word].tolist() # 현재 단어의 임베딩 값을 가져온다
        print('data_pre:', data) # 현재 단어의 임베딩 값을 출력해본다

        data.insert(0, word) # 시작 부분에 해당 단어를 넣는다
        print('data_after:', data) # 현재 단어의 이름과 함께 임베딩 값을 출력해본다

        for item in data: # 단어 이름부터 시작하여 각 벡터의 값을 저장한다
            f.write("%s " % item)
        f.write("\n")

In [ ]:
#Word2Vec – 유사어 찾기
model = Word2Vec.load('embedding_window10_mincnt5_skipgram.model')
print('--- 유사단어 출력 ---')
similarWords = model.wv.most_similar(positive=['행복', '웃음', '밝'], topn=3)
print(similarWords) # [('흐뭇', 0.8801068067550659), ('마음', 0.8738192319869995), ('해맑', 0.8424177169799805)]

word = []
for similarWord in similarWords: # 유사도값을 제외하고 단어만 모은다
    word.append(similarWord[0])
print(word) # ['흐뭇', '마음', '해맑']


--- 유사단어 출력 ---
[('일어서', 0.9741085171699524), ('깜짝', 0.9731009602546692), ('따뜻하', 0.972411572933197)]
['일어서', '깜짝', '따뜻하']


In [ ]:
#Word2Vec – 두 단어 사이의 유사도 계산
print('--- 두 단어의 유사도 계산 ---')
print('한국과 일본:', model.wv.similarity('한국', '일본'))
print('한국과 미국:', model.wv.similarity('한국', '미국'))
print('한국과 중국:', model.wv.similarity('한국', '중국'))

--- 두 단어의 유사도 계산 ---
한국과 일본: 0.6982431
한국과 미국: 0.75216246
한국과 중국: 0.7944958
